<style>
@font-face {
  font-family: CharisSILW;
  src: url(files/CharisSIL-R.woff);
}
@font-face {
  font-family: CharisSILW;
  font-style: italic;
  src: url(files/CharisSIL-I.woff);
}
@font-face {
	font-family: CharisSILW;
	font-weight: bold;
	src: url(files/CharisSIL-B.woff);
}
@font-face {
	font-family: CharisSILW;
	font-weight: bold;
	font-style: italic;
	src: url(files/CharisSIL-BI.woff);
}

div.cell, div.text_cell_render{
    max-width:1000px;
}

h1 {
    text-align:center;
    font-family: Charis SIL, CharisSILW, serif;
}

.rendered_html {
    font-size: 130%;
    line-height: 1.3;
}

.rendered_html li {
    line-height: 2;
}

.rendered_html h1{
    line-height: 1.3;
}

.rendered_html h2{
    line-height: 1.2;
}

.rendered_html h3{
    line-height: 1.0;
}

.text_cell_render {
    font-family: Charis SIL, CharisSILW, serif;
    line-height: 145%;
}

li li {
    font-size: 85%;
}
</style>

## Project overview

The objective here is to predict African wells that are non-functional or in need of repair.  

## First Draft of an Analysis


In [1]:
import pandas as pd
import numpy as np
  
features_df = pd.DataFrame.from_csv("well_data.csv")
labels_df   = pd.DataFrame.from_csv("well_labels.csv")  
labels_df.head(20)

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional
9944,functional
19816,non functional
54551,non functional
53934,non functional


In [2]:
features_df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,3/14/11,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,3/6/13,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2/25/13,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,1/28/13,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,7/13/11,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


### Transforming string labels into integers

In [3]:
def label_map(y):
    if y=="functional":
        return 2
    elif y=="functional needs repair":
        return 1
    else:
        return 0
labels_df = labels_df.applymap(label_map)
labels_df.head()

,status_group
id,
69572,2
8776,2
34310,2
67743,0
19728,2


### Transforming string features into integers

In [4]:
def transform_feature(df, column_name):

    unique_values = set(df[column_name].tolist())
    transformer_dict = {}
    for ii, value in enumerate(unique_values):
        transformer_dict[value] = ii
        
    def label_map(y):
        return transformer_dict[y]
    
    df[column_name] = df[column_name].apply(label_map)
    return df

columns_to_transform = ["funder", "installer", "wpt_name", "basin", "subvillage",
                    "region", "lga", "ward", "public_meeting", "recorded_by",
                    "scheme_management", "scheme_name", "permit",
                    "extraction_type", "extraction_type_group",
                    "extraction_type_class",
                    "management", "management_group",
                    "payment", "payment_type",
                    "water_quality", "quality_group", "quantity", "quantity_group",
                    "source", "source_type", "source_class",
                    "waterpoint_type", "waterpoint_type_group"]
for column in columns_to_transform:
    features_df = transform_feature(features_df, column)
    
if "date_recorded" in features_df.columns.values:
    features_df.drop("date_recorded", axis=1, inplace=True)
    
features_df.head(10)

,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,1539,1390,1750,34.938093,-9.856322,15206,0,5,9411,...,4,0,0,2,2,1,1,1,1,1
8776,0.0,774,1399,136,34.698766,-2.147466,5616,0,7,3481,...,5,0,0,1,1,6,4,2,1,1
34310,25.0,906,686,1401,37.460664,-3.821329,6144,0,6,11383,...,1,0,0,2,2,7,5,2,4,1
67743,0.0,590,263,556,38.486161,-11.155298,1252,0,1,8125,...,5,0,0,0,0,2,6,1,4,1
19728,0.0,1342,0,537,31.130847,-1.825359,29686,0,7,18664,...,5,0,0,3,3,6,4,2,1,1
9944,20.0,1303,0,2132,39.172796,-4.765587,15053,0,6,13835,...,1,5,3,2,2,5,3,0,4,1
19816,0.0,1632,0,1342,33.362410,-3.766365,4408,0,3,544,...,5,0,0,2,2,2,6,1,0,0
54551,0.0,327,0,2132,32.620617,-4.226198,15542,0,8,2793,...,3,1,4,2,2,4,2,1,0,0
53934,0.0,787,0,529,32.711100,-5.146712,27117,0,8,5234,...,5,5,3,3,3,2,6,1,0,0


Ok, a couple last steps to get everything ready for sklearn.  The features and labels are taken out of their dataframes and put into a numpy.ndarray and list, respectively.  

In [5]:
X = features_df
y = labels_df["status_group"]

### Predicting well failures with logistic regression

In [6]:
import sklearn.linear_model
import sklearn.cross_validation

clf = sklearn.linear_model.LogisticRegression()
score = sklearn.cross_validation.cross_val_score(clf, X, y)
print(score)

C:\Users\tarkan\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[ 0.68767677  0.68393939  0.68378788]


### Comparing logistic regression to tree-based methods

In [7]:
import sklearn.tree
import sklearn.ensemble

clf = sklearn.tree.DecisionTreeClassifier()
score = sklearn.cross_validation.cross_val_score(clf, X, y)
print(score)

clf = sklearn.ensemble.RandomForestClassifier()
score = sklearn.cross_validation.cross_val_score(clf, X, y)
print(score)

[ 0.73666667  0.73752525  0.73565657]
[ 0.78883838  0.78848485  0.78823232]


## Paying down technical debt and tuning the models

### One-hot encoding to make dummy variables

In [9]:
import sklearn.preprocessing

def hot_encoder(df, column_name):
    print("one-hot encoding {}".format(column_name))
    column = df[column_name].tolist()
    
    column = np.reshape(column, (len(column), 1))
    
    enc = sklearn.preprocessing.OneHotEncoder()
    enc.fit(column)
    new_column = enc.transform(column).toarray()
    
    column_titles = []
    for ii in range( len(new_column[0]) ):
        this_column_name = column_name+" "+str(ii)
        df[this_column_name] = new_column[:,ii]
    
    df.drop(column_name, axis=1, inplace=True)
    return df

In [10]:
print(features_df.columns.values)

if "funder" in features_df.columns.values:
    features_df.drop("funder", axis=1, inplace=True)
    columns_to_transform.remove("funder")
if "installer" in features_df.columns.values:   
    features_df.drop("installer", axis=1, inplace=True)
    columns_to_transform.remove("installer")        
if "wpt_name" in features_df.columns.values:
    features_df.drop("wpt_name", axis=1, inplace=True)
    columns_to_transform.remove("wpt_name")
if "subvillage" in features_df.columns.values:
    features_df.drop("subvillage", axis=1, inplace=True)
    columns_to_transform.remove("subvillage")
if "ward" in features_df.columns.values:
    features_df.drop("ward", axis=1, inplace=True)
    columns_to_transform.remove("ward")
if "basin" in features_df.columns.values:
    features_df.drop("basin", axis=1, inplace=True)
    columns_to_transform.remove("basin")
if "scheme_name" in features_df.columns.values:
    features_df.drop("scheme_name", axis=1, inplace=True)
    columns_to_transform.remove("scheme_name")

for feature in columns_to_transform:
    features_df = hot_encoder(features_df, feature)
    
features_df.head()

['amount_tsh' 'funder' 'gps_height' 'installer' 'longitude' 'latitude'
 'wpt_name' 'num_private' 'basin' 'subvillage' 'region' 'region_code'
 'district_code' 'lga' 'ward' 'population' 'public_meeting' 'recorded_by'
 'scheme_management' 'scheme_name' 'permit' 'construction_year'
 'extraction_type' 'extraction_type_group' 'extraction_type_class'
 'management' 'management_group' 'payment' 'payment_type' 'water_quality'
 'quality_group' 'quantity' 'quantity_group' 'source' 'source_type'
 'source_class' 'waterpoint_type' 'waterpoint_type_group']
one-hot encoding region
one-hot encoding lga
one-hot encoding public_meeting
one-hot encoding recorded_by
one-hot encoding scheme_management
one-hot encoding permit
one-hot encoding extraction_type
one-hot encoding extraction_type_group
one-hot encoding extraction_type_class
one-hot encoding management
one-hot encoding management_group
one-hot encoding payment
one-hot encoding payment_type
one-hot encoding water_quality
one-hot encoding quality_grou

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,region 0,...,waterpoint_type 3,waterpoint_type 4,waterpoint_type 5,waterpoint_type 6,waterpoint_type_group 0,waterpoint_type_group 1,waterpoint_type_group 2,waterpoint_type_group 3,waterpoint_type_group 4,waterpoint_type_group 5
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,1390,34.938093,-9.856322,0,11,5,109,1999,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8776,0.0,1399,34.698766,-2.147466,0,20,2,280,2010,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34310,25.0,686,37.460664,-3.821329,0,21,4,250,2009,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
67743,0.0,263,38.486161,-11.155298,0,90,63,58,1986,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19728,0.0,0,31.130847,-1.825359,0,18,1,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
X = features_df.as_matrix()
y = labels_df["status_group"].tolist()

clf = sklearn.ensemble.RandomForestClassifier()
score = sklearn.cross_validation.cross_val_score(clf, X, y)
print(score)

[ 0.78181818  0.78575758  0.7770202 ]


## Pipeline and GridSearchCV

In [12]:
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=100)
selected_X = select.fit_transform(X, y)

print(selected_X.shape)

(59400L, 100L)


C:\Users\tarkan\Anaconda2\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [158] are constant.
  UserWarning)
C:\Users\tarkan\Anaconda2\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


### Pipeline

In [13]:
import sklearn.pipeline

select = sklearn.feature_selection.SelectKBest(k=100)
clf = sklearn.ensemble.RandomForestClassifier()

steps = [('feature_selection', select),
         ('random_forest', clf)]

pipeline = sklearn.pipeline.Pipeline(steps)

X_train, X_test, y_train, y_test = sklearn.cross_validation.train_test_split(X, y, test_size=0.33, random_state=42)

# fit your pipeline on X_train and y_train
pipeline.fit( X_train, y_train )
# call pipeline.predict() on your X_test data to make a set of test predictions
y_prediction = pipeline.predict( X_test )
# test your predictions using sklearn.classification_report()
report = sklearn.metrics.classification_report( y_test, y_prediction )
# and print the report
print(report)

             precision    recall  f1-score   support

          0       0.78      0.78      0.78      7458
          1       0.42      0.36      0.39      1425
          2       0.81      0.82      0.82     10719

avg / total       0.77      0.78      0.77     19602



### GridSearchCV

In [14]:
import sklearn.grid_search

parameters = dict(feature_selection__k=[100, 200], 
              random_forest__n_estimators=[50, 200],
              random_forest__min_samples_split=[2, 3, 4])

cv = sklearn.grid_search.GridSearchCV(pipeline, param_grid=parameters, verbose=True)

cv.fit(X_train, y_train)
y_predictions = cv.predict(X_test)
report = sklearn.metrics.classification_report(y_test, y_predictions)
print(report)


C:\Users\tarkan\Anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\tarkan\Anaconda2\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [158 160] are constant.
  UserWarning)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


C:\Users\tarkan\Anaconda2\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [ 71 158] are constant.
  UserWarning)
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  4.8min finished


             precision    recall  f1-score   support

          0       0.81      0.78      0.79      7458
          1       0.50      0.35      0.41      1425
          2       0.81      0.87      0.84     10719

avg / total       0.79      0.79      0.79     19602



In [15]:
print cv.best_estimator_ 

Pipeline(steps=[('feature_selection', SelectKBest(k=200, score_func=<function f_classif at 0x000000000A8F8978>)), ('random_forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=...mators=200, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])
